# Install, Paths and Parameters

In [21]:
import os
from pathlib import Path
import getpass
import numpy as np
import time
import torch
from torch import nn
from tqdm import tqdm
import random
import sys

# allow imports when running script from within project dir
[sys.path.append(i) for i in ['.', '..']]

# local
from src.helpers.helpers import get_random_indexes, get_random_classes
from src.model.dino_model import get_dino, ViTWrapper
from src.model.data import create_loader
from src.model.eval import validate_network

# Custom imports
import torchattacks
from torchattacks import *
import torch.optim as optim
from torchvision import transforms as pth_transforms

# seed
SEED = 42
random.seed(SEED)
torch.manual_seed(SEED)
np.random.seed(SEED)

username = getpass.getuser()
DATA_PATH = Path('/','cluster', 'scratch', 'thobauma', 'dl_data')

DN_PATH = Path(DATA_PATH, 'damageNet')
DN_LABEL_PATH = Path(DN_PATH, 'val_damagenet.txt')
DN_IMAGES_PATH = Path(DN_PATH, 'images')

ORI_PATH = Path(DATA_PATH, 'ori_data/validation')
ORI_LABEL_PATH = Path(ORI_PATH,'correct_labels.txt')
ORI_IMAGES_PATH = Path(ORI_PATH,'images')

In [22]:
INDEX_SUBSET = get_random_indexes()

BATCH_SIZE = 12

DEVICE = 'cuda'

In [23]:
len(INDEX_SUBSET)

1000

In [24]:
#!python ../setup/collect_env.py

In [25]:
model, linear_classifier = get_dino(model_name='vit_base', patch_size=16, n_last_blocks=1, avgpool_patchtokens=True, device=DEVICE)

Please use the `--pretrained_weights` argument to indicate the path of the checkpoint to evaluate.
Since no pretrained weights have been provided, we load the reference pretrained DINO weights.
Model vit_base built.
Embed dim 1536
We load the reference pretrained linear weights from dino_vitbase16_pretrain/dino_vitbase16_linearweights.pth.


# Load data

In [26]:
org_loader = create_loader(ORI_IMAGES_PATH, ORI_LABEL_PATH, INDEX_SUBSET, None, BATCH_SIZE, is_adv_training=True)
len(org_loader.dataset)

PermissionError: [Errno 13] Permission denied: '/cluster/scratch/thobauma/dl_data/ori_data/validation/correct_labels.txt'

# Wrap model

In [27]:
model_wrap = ViTWrapper(model, linear_classifier, DEVICE, n_last_blocks=1, avgpool_patchtokens=True)
model_wrap= model_wrap.to(DEVICE)

# Attack model

In [8]:
attacks = [
    FGSM(model_wrap, eps=0.062),
    CW(model_wrap, c=50, lr=0.0031, steps=30),
    PGD(model_wrap, eps=0.062, alpha=0.031, steps=20)
]

#### Compute clean accuracy

In [13]:
clean_correct = 0
total = len(org_loader.dataset)
start = time.time()

for images, labels, _ in tqdm(org_loader):
    
    cuda_images = images.to(DEVICE)
    clean_outputs = model_wrap(cuda_images)
    labels = labels.to(DEVICE)

    _, pre_clean = torch.max(clean_outputs.data, 1)

    clean_correct += (pre_clean == labels).sum()

print('Total elapsed time (sec): %.2f' % (time.time() - start))
print('Clean accuracy: %.2f %%' % (100 * float(clean_correct) / total))

  0%|          | 0/84 [00:00<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: '/cluster/scratch/thobauma/dl_data/ori_data/images/ILSVRC2012_val_00033554.JPEG'

In [9]:
torch.cuda.empty_cache()

In [12]:
for atk in attacks:
    
    print("-"*70)
    print(atk)
    
    correct = 0
    clean_correct = 0
    start = time.time()
    
    for images, labels, _ in tqdm(org_loader):
        
        cuda_images = images.to(DEVICE)
        labels = labels.to(DEVICE)
        
        adv_images = atk(images, labels)
        outputs = model_wrap(adv_images)

        _, pre = torch.max(outputs.data, 1)

        correct += (pre == labels).sum()

        del images
        del labels
        torch.cuda.empty_cache()

    print('Total elapsed time (sec): %.2f' % (time.time() - start))
    print('Accuracy against attack: %.2f %%' % (100 * float(correct) / total))

  0%|          | 0/84 [00:00<?, ?it/s]

----------------------------------------------------------------------
FGSM(model_name=ViTWrapper, device=cuda:0, eps=0.062, attack_mode=default, return_type=float)


 85%|████████▍ | 71/84 [00:28<00:05,  2.51it/s]


FileNotFoundError: [Errno 2] No such file or directory: '/cluster/scratch/thobauma/dl_data/ori_data/images/ILSVRC2012_val_00048990.JPEG'